In [1]:

#!pip install bs4
#!pip install selenium
#!pip install pickle
#!pip install selenium

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
import selenium
from selenium import webdriver
from tqdm import tqdm as tqdmn
from selenium.webdriver.common.by import By
import pandas as pd

Url_With_Coordinates = []
option = webdriver.ChromeOptions()
driver = webdriver.Chrome("C:\\chromedriver.exe", options=option)

prefs = {'profile.default_content_setting_values': {'images':2, 'javascript':2}}#speed up by not loading images and JavaScript

C:\Users\HoangLe.HoangLeLaptop\AppData\Local\Temp\ipykernel_11572\589117858.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\\chromedriver.exe", options=option)


In [3]:
success_test_street = pd.read_excel("data/address_require_review.xlsx")
success_test_street['Address'].replace('', np.nan, inplace=True)
success_test_street.dropna(subset=['Address'], inplace=True)

#The special character "/" can broke the URL, so put it as space
success_test_street.Address = success_test_street.Address.str.replace('/', ' ')
success_test_street.head(20)

,Hospital,Address
1,Bệnh viện Hồng Ngọc,"55, Yên Ninh, Ba Đình, Hà Nội"
2,Bệnh viện Mắt Quốc tế DND,"126-128 Bùi Thị Xuân, P. Bùi Thị Xuân, Quận Ha..."
3,Bệnh viện Đa Khoa Quốc tế Vinmec – Hà Nội,"458 Minh Khai, P.Vĩnh Tuy, Q.Hai Bà Trưng, Hà Nội"
5,Bệnh viện MEDELAB Việt Nam,"86-88 Nguyễn Lương Bằng, Nam Đồng, Đống Đa, Hà..."
6,Bệnh viên Đa khoa Quốc tế THU CÚC,"286 Thụy Khuê, Phường Bưởi, Quận Tây Hồ, Hà Nội"
7,Bệnh viện Đa khoa tư nhân 16A Hà Đông,"150 khu giãn dân Yên Phúc, Phúc La, Hà Đông, H..."
8,Bệnh viện Đa khoa Tâm Anh,"30A Lý Nam Đế, Phường Cửa Đông, Quận Hòa Kiếm, HN"
9,Bệnh viện Đa khoa Melatec,"42-44 Nghĩa Dung, P. Phúc xá, Q.Ba Đình, Hà Nội"
10,Bệnh viện Hữu nghị Việt Đức,"Số 40 Tràng Thi, phường Hàng Bông, quận Hoàn K..."
11,Bệnh viện Hà Thành,Số 61 phố vũ Thạnh ô chợ dừa đống đa hà nội


Waiting time will depend on your browser speed. Edited your desire waiting time here. The program set the waiting time to be 4 seconds for each line of address.

In [4]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException  # Import NoSuchElementException

##CHange waiting time in order to finish loading the page. Currently 4 seconds 
wait = WebDriverWait(driver, 4)

First round of scraping. The program will pull data if Google Maps can appear the location immediately after searching for the facilities' names, or facilities' addresses, or combination of both.

This round take approx: 30.min for every 30 line of addresses.

Then, filter out addresses needs further rounds of scraping.

In [5]:
success_google_map_address = []

#change to 1 seconds only
wait = WebDriverWait(driver, 1)
for index, row in success_test_street.iterrows():
    try:
        driver.get("https://www.google.com/maps/search/"+row["Address"])
        element = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[7]/div[3]/button/div/div[3]/div[1]')))
        # Now that the element is guaranteed to be present, extract its text
        temp = element.text
        success_google_map_address.append(temp)

    except:
        try:
            driver.get("https://www.google.com/maps/search/"+row["Hospital"])
            element = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[7]/div[3]/button/div/div[3]/div[1]')))
            # Now that the element is guaranteed to be present, extract its text
            temp = element.text
            success_google_map_address.append(temp)
        except:
            try:
                driver.get("https://www.google.com/maps/search/"+row["Address"] + " " + row["Hospital"])
                element = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[7]/div[3]/button/div/div[3]/div[1]')))
                # Now that the element is guaranteed to be present, extract its text
                temp = element.text
                success_google_map_address.append(temp)
            except:
                success_google_map_address.append("Can't find location index: ")

#address that's empty or need review
broken_string_indices = [i for i, s in enumerate(success_google_map_address) if s == "" or s.startswith("Can't find location index: ")]
selected_rows = success_test_street.iloc[broken_string_indices]

print("Detected: ", len(success_google_map_address) - len(broken_string_indices), " addresses in this first round.")
broken_string_indices
selected_rows

Detected:  13  addresses in this first round.


,Hospital,Address
1,Bệnh viện Hồng Ngọc,"55, Yên Ninh, Ba Đình, Hà Nội"
2,Bệnh viện Mắt Quốc tế DND,"126-128 Bùi Thị Xuân, P. Bùi Thị Xuân, Quận Ha..."
3,Bệnh viện Đa Khoa Quốc tế Vinmec – Hà Nội,"458 Minh Khai, P.Vĩnh Tuy, Q.Hai Bà Trưng, Hà Nội"
6,Bệnh viên Đa khoa Quốc tế THU CÚC,"286 Thụy Khuê, Phường Bưởi, Quận Tây Hồ, Hà Nội"
7,Bệnh viện Đa khoa tư nhân 16A Hà Đông,"150 khu giãn dân Yên Phúc, Phúc La, Hà Đông, H..."
...,...,...
319,Bệnh viện Parkway East,321 Joo Chiat Place Singapore\n427990
320,Bệnh viện Mount Elizabeth Novena,38 Irrawaddy Road Singapore 329563
322,Bệnh viện Piyavate,"998 Rimklongsamsen Road, Bangkapi, Huaywang, B..."
323,Bệnh viện Vejthani,"1 Ladprao Road 111, Klong-Chan\nBangkapi, Bang..."


Saving the results to pickle file.

In [6]:
import pickle
# Saving the object
with open('pickle/success_google_map_address.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(success_google_map_address, f)

#Loadinf object
success_google_map_address = pickle.load(open("pickle/success_google_map_address.pkl","rb"))

Second round of scraping. Some addresses needs further time waiting and second search to be recognized. This round will take Approx: 20 min for every 30 lines of missing addresses.

In [7]:
renewed_first = []

#change back to default time
wait = WebDriverWait(driver, 3)
for index, row in selected_rows.iterrows():
    try:
        driver.get("https://www.google.com/maps/search/"+row["Address"]+ " " + row["Hospital"])
        element = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[7]/div[3]/button/div/div[3]/div[1]')))
        # Now that the element is guaranteed to be present, extract its text
        temp = element.text
        renewed_first.append(temp)
    except:
        renewed_first.append("Can't find after second round")

#address that's empty or need review
broken_string_indices_second_round = [i for i, s in enumerate(renewed_first) if s == "" or s.startswith("Can't find after second round")]
selected_rows_2nd = success_test_street.iloc[broken_string_indices_second_round]

print("Detected additional: ", len(renewed_first) - len(broken_string_indices_second_round), " addresses in this second round.")
selected_rows_2nd


Detected additional:  0  addresses in this second round.


,Hospital,Address
1,Bệnh viện Hồng Ngọc,"55, Yên Ninh, Ba Đình, Hà Nội"
2,Bệnh viện Mắt Quốc tế DND,"126-128 Bùi Thị Xuân, P. Bùi Thị Xuân, Quận Ha..."
3,Bệnh viện Đa Khoa Quốc tế Vinmec – Hà Nội,"458 Minh Khai, P.Vĩnh Tuy, Q.Hai Bà Trưng, Hà Nội"
5,Bệnh viện MEDELAB Việt Nam,"86-88 Nguyễn Lương Bằng, Nam Đồng, Đống Đa, Hà..."
6,Bệnh viên Đa khoa Quốc tế THU CÚC,"286 Thụy Khuê, Phường Bưởi, Quận Tây Hồ, Hà Nội"
...,...,...
298,Khu B - Bệnh viện Đồng Nai 2,"02, Đồng Khởi, Phường Tam Hòa, TP Biên Hòa"
300,Phòng khám Đa khoa Y Sài Gòn,"2 8, Khu phố 6, Phường Tam Hiệp,\nTP. Biên Hòa"
301,Nha khoa Vạn Thành,"Chi nhánh Biên Hòa\n17 11- CMT8, P.Quang Vinh,..."
302,Nha khoa Vạn Thành,"Chi nhánh Thống Nhất\n14D Phúc Nhạc, Gia Tân 3..."


In [8]:

renewed = []

wait = WebDriverWait(driver, 3)
for index, row in selected_rows_2nd.iterrows():
    try:
        driver.get("https://www.google.com/maps/search/"+row["Address"])
        element = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[9]/div[3]/button/div/div[3]/div[1]')))
        temp = element.text
        renewed.append(temp)
    except:
        try:
            driver.get("https://www.google.com/maps/search/"+row["Hospital"])
            element = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[7]/div[3]/button/div/div[3]/div[1]')))
            temp = element.text
            renewed.append(temp)
        except:
            try:
                #print(str(index))
                driver.get("https://www.google.com/maps/search/"+row["Address"] + " " + row["Hospital"])
                element = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[7]/div[3]/button/div/div[3]/div[1]')))
                # Now that the element is guaranteed to be present, extract its text
                temp = element.text
                renewed.append(temp)
            except:
                renewed.append("still Can't find location index: ")
                #print(str(index))

#Final round of review
temp_final = [i for i, s in enumerate(renewed) if s == "" or s.startswith("still Can't find location index: ")]
selected_rows_final = selected_rows_2nd.iloc[temp_final]
selected_rows_final

print("Detected finally: ", len(renewed) - len(temp_final), " addresses in this third round.")


Detected finally:  0  addresses in this third round.


Saving the results to pickle file.

In [9]:
# Saving the object
with open('pickle/renewed_address.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(renewed, f)

#Loading object
with open("pickle/renewed_address.pkl","rb") as f:
    renewed = pickle.load(f)

In [11]:
remaining_address = []

for index, row in selected_rows_final.iterrows():
    driver.get("https://www.google.com/maps/search/"+row["Address"] + " " + row["Hospital"])

    for divnum in range (1,4):
        try:
            # Attempt to navigate to the URL and scrape data
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div:nth-child('+str(divnum)+') > div > a'))).click()
    
            try: 
                gurus = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div:nth-child(7) > div:nth-child(3) > button > div > div.rogA2c > div.Io6YTe.fontBodyMedium'))).text
            except:
                gurus = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div:nth-child(9) > div:nth-child(3) > button > div > div.rogA2c > div.Io6YTe.fontBodyMedium'))).text
                                                                                   
            remaining_address.append("(edited)"+ gurus)
            break  # If successful, break the loop
        
        except NoSuchElementException:
            # If the element wasn't found, print a message and continue with the next URL
            print(f"Element not found in divnum = {divnum}. Trying the next divnum.")
        
        except Exception as e:
            # If another type of exception occurred, print a message and continue with the next URL
            print(f"An error occurred. print Trying the next URL.")

#Final round of review

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.90)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x003DA813+48355]
	(No symbol) [0x0036C4B1]
	(No symbol) [0x00275358]
	(No symbol) [0x0025D293]
	(No symbol) [0x002BE37B]
	(No symbol) [0x002CC473]
	(No symbol) [0x002BA536]
	(No symbol) [0x002982DC]
	(No symbol) [0x002993DD]
	GetHandleVerifier [0x0063AABD+2539405]
	GetHandleVerifier [0x0067A78F+2800735]
	GetHandleVerifier [0x0067456C+2775612]
	GetHandleVerifier [0x004651E0+616112]
	(No symbol) [0x00375F8C]
	(No symbol) [0x00372328]
	(No symbol) [0x0037240B]
	(No symbol) [0x00364FF7]
	BaseThreadInitThunk [0x76467D59+25]
	RtlInitializeExceptionChain [0x7772B74B+107]
	RtlClearBits [0x7772B6CF+191]


In [ ]:
remaining_address

['(edited)1-3 Đ. Trịnh Văn Cấn, Phường Cầu Ông Lãnh, Quận 1, Thành phố Hồ Chí Minh, Vietnam',
 '(edited)31 Nguyễn Đình Chiểu, Đa Kao, Quận 1, Thành phố Hồ Chí Minh 700000, Vietnam',
 '(edited)1-3 Đ. Trịnh Văn Cấn, Phường Cầu Ông Lãnh, Quận 1, Thành phố Hồ Chí Minh, Vietnam',
 '(edited)Bitexco Financial Tower, 3rd, 2 Hải Triều, St, Quận 1, Thành phố Hồ Chí Minh, Vietnam',
 '(edited)45 Đ. An D. Vương, P.8, Quận 5, Thành phố Hồ Chí Minh 700000, Vietnam',
 '(edited)244A Đ. Cống Quỳnh, Phường Phạm Ngũ Lão, Quận 1, Thành phố Hồ Chí Minh, Vietnam',
 '(edited)463A Đ. Hồng Bàng, Phường 15, Quận 5, Thành phố Hồ Chí Minh, Vietnam',
 '(edited)1298 Huỳnh Tấn Phát, Phú Mỹ, Quận 7, Thành phố Hồ Chí Minh, Vietnam',
 '(edited)398 Đ. 3 Tháng 2, P.12, Quận 10, Thành phố Hồ Chí Minh 700000, Vietnam']

Import to CSV file

In [ ]:
import csv

#Type in your desire path
with open('edited_address.csv', 'w') as file:
    wr = csv.writer(file)
    wr.writerow(Url_With_Coordinates)